In [1]:
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
#from sklearn.model_selection import train_test_split

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [3]:
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  163780
Total Vocab:  58


In [4]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163680


In [5]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [29]:
# define the LSTM model
model = Sequential()

model.add(LSTM(300, input_shape=(X.shape[1], X.shape[2]), dropout_U = 0.2, dropout_W = 0.2, kernel_initializer='he_uniform', return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(300,  dropout_U = 0.2, dropout_W = 0.2, kernel_initializer='he_uniform', return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(300,  dropout_U = 0.2, dropout_W = 0.2, kernel_initializer='he_uniform', return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(300,  dropout_U = 0.2, dropout_W = 0.2, kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dense(100, kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, input_shape=(100, 1), kernel_initializer="he_uniform", return_sequences=True, dropout=0.2, recurrent_dropout=0.2)`
  after removing the cwd from sys.path.
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, kernel_initializer="he_uniform", return_sequences=True, dropout=0.2, recurrent_dropout=0.2)`
  
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, kernel_initializer="he_uniform", return_sequences=True, dropout=0.2, recurrent_dropout=0.2)`
  
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, kernel_initializer="he_uniform", dropout=0.2, recurrent_dropout=0.2)`
  # Remove the CWD from sys.path while we load stuff.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_57 (LSTM)               (None, 100, 300)          362400    
_________________________________________________________________
batch_normalization_57 (Batc (None, 100, 300)          1200      
_________________________________________________________________
lstm_58 (LSTM)               (None, 100, 300)          721200    
_________________________________________________________________
batch_normalization_58 (Batc (None, 100, 300)          1200      
_________________________________________________________________
lstm_59 (LSTM)               (None, 100, 300)          721200    
_________________________________________________________________
batch_normalization_59 (Batc (None, 100, 300)          1200      
_________________________________________________________________
lstm_60 (LSTM)               (None, 300)               721200    
__________

In [23]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [24]:
history=model.fit(X, y, batch_size=50, epochs=20, callbacks=callbacks_list)

Epoch 1/17
   800/163680 [..............................] - ETA: 831:30:49 - loss: 4.1897 - acc: 0.1187

KeyboardInterrupt: 

import matplotlib.pyplot as plt
plt.plot(history.history['acc'])

plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

In [ ]:
filename = "weights-improvement-30-1.0637-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.save('LSTM_model.h5')